In [10]:
# Importar librerías
import pandas as pd
from sqlalchemy import create_engine

# Configuración de la base de datos
db_config = {
    'user': 'practicum_student',  # username
    'pwd': 'QnmDH8Sc2TQLvy2G3Vvh7',  # password
    'host': 'yp-trainers-practicum.cluster-czs0gxyx2d8w.us-east-1.rds.amazonaws.com',
    'port': 5432,  # puerto
    'db': 'data-analyst-final-project-db'  # nombre de la base de datos
}

# Cadena de conexión
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db']
)

# Crear motor de conexión
engine = create_engine(connection_string, connect_args={'sslmode': 'require'})

# Probar la conexión con una consulta sencilla
pd.io.sql.read_sql("SELECT 1;", con=engine)

,?column?
0,1


In [14]:
pd.read_sql("SELECT * FROM books LIMIT 5;", con=engine)


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [15]:
pd.read_sql("SELECT * FROM authors LIMIT 5;", con=engine)


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [16]:
pd.read_sql("SELECT * FROM publishers LIMIT 5;", con=engine)


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [17]:
pd.read_sql("SELECT * FROM ratings LIMIT 5;", con=engine)


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [18]:
pd.read_sql("SELECT * FROM reviews LIMIT 5;", con=engine)


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Encuentra el número de libros publicados después del 1 de enero de 2000.


In [22]:
import pandas as pd

In [24]:
query_1 = """
SELECT COUNT(*) AS books_after_2000
FROM books
WHERE publication_date > '2000-01-01';
"""

pd.read_sql(query_1, con=engine)


,books_after_2000
0,819


There are 819 books published after January 1, 2000.
This shows that a large part of the catalog is made up of recent books, which helps understand modern reading trends.

## Encuentra el número de reseñas de usuarios y la calificación promedio para cada libro.


In [25]:
query_2 = """
SELECT 
    b.book_id,
    b.title,
    COUNT(DISTINCT rv.review_id) AS total_reviews,
    AVG(rt.rating) AS avg_rating
FROM books b
LEFT JOIN reviews rv ON b.book_id = rv.book_id
LEFT JOIN ratings rt ON b.book_id = rt.book_id
GROUP BY b.book_id, b.title
ORDER BY avg_rating DESC;
"""

df_book_reviews_ratings = pd.read_sql(query_2, con=engine)
df_book_reviews_ratings

,book_id,title,total_reviews,avg_rating
0,86,Arrows of the Queen (Heralds of Valdemar #1),2,5.00
1,901,The Walking Dead Book One (The Walking Dead #...,2,5.00
2,390,Light in August,2,5.00
3,972,Wherever You Go There You Are: Mindfulness Me...,2,5.00
4,136,Captivating: Unveiling the Mystery of a Woman'...,2,5.00
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,3,2.25
996,316,His Excellency: George Washington,2,2.00
997,202,Drowning Ruth,3,2.00
998,371,Junky,2,2.00


Most books have 2–3 reviews, with average ratings ranging from 1.5 to 5.0.
This shows that while many books are rated, there are few written reviews per book.
Books with higher average ratings could be recommended or highlighted.

## Identifica la editorial que ha publicado el mayor número de libros con más de 50 páginas 

In [26]:
query_3 = """
SELECT 
    p.publisher,
    COUNT(*) AS total_books
FROM books b
JOIN publishers p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher
ORDER BY total_books DESC
LIMIT 1;
"""

df_top_publisher = pd.read_sql(query_3, con=engine)
df_top_publisher

,publisher,total_books
0,Penguin Books,42


Penguin Books has published the most substantial books over 50 pages, with 42 titles.
This suggests Penguin is a leading publisher and its books could be prioritized for quality analysis or recommendations.

## Identifica al autor que tiene la más alta calificación promedio del libro: mira solo los libros con al menos 50 calificaciones.


In [29]:
query_4 = """
WITH book_ratings AS (
    SELECT
        b.book_id,
        b.author_id,
        COUNT(r.rating) AS num_ratings,
        AVG(r.rating) AS avg_rating
    FROM books b
    JOIN ratings r ON b.book_id = r.book_id
    GROUP BY b.book_id, b.author_id
    HAVING COUNT(r.rating) >= 50
)
SELECT 
    a.author,
    AVG(br.avg_rating) AS author_avg_rating
FROM book_ratings br
JOIN authors a ON br.author_id = a.author_id
GROUP BY a.author
ORDER BY author_avg_rating DESC
LIMIT 1;
"""

df_top_author = pd.read_sql(query_4, con=engine)
df_top_author


,author,author_avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844


The author with the highest average rating is J.K. Rowling/Mary GrandPré, with 4.28 (for books with at least 50 ratings).
This shows their work is consistently well-received and could be highlighted for recommendations or engagement.

## Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.


In [30]:
query_5 = """
WITH active_users AS (
    SELECT username
    FROM ratings
    GROUP BY username
    HAVING COUNT(*) > 50
),
user_review_counts AS (
    SELECT 
        au.username,
        COUNT(rv.review_id) AS review_count
    FROM active_users au
    LEFT JOIN reviews rv ON au.username = rv.username
    GROUP BY au.username
)
SELECT AVG(review_count) AS avg_reviews_per_active_user
FROM user_review_counts;
"""

df_avg_reviews = pd.read_sql(query_5, con=engine)
df_avg_reviews

,avg_reviews_per_active_user
0,24.333333


Users who rated over 50 books write an average of 24.33 reviews.
This shows they are very active, and their opinions could be key for recommendations or analyzing reading trends.